In [ ]:
import pandas as pd

# Reading all the text files
df_agency = pd.read_csv('0-data/gtfs23Sept/agency.txt', sep=",")
df_calendar = pd.read_csv('0-data/gtfs23Sept/calendar.txt', sep=",")
df_calendar_dates = pd.read_csv('0-data/gtfs23Sept/calendar_dates.txt', sep=",")
df_routes = pd.read_csv('0-data/gtfs23Sept/routes.txt', sep=",")
df_shapes = pd.read_csv('0-data/gtfs23Sept/shapes.txt', sep = ",")
df_stop_times = pd.read_csv('0-data/gtfs23Sept/stop_times.txt', sep = ",")
df_stops = pd.read_csv('0-data/gtfs23Sept/stops.txt', sep = ",")
df_translations = pd.read_csv('0-data/gtfs23Sept/translations.txt', sep = ",")
df_trips = pd.read_csv('0-data/gtfs23Sept/trips.txt', sep = ",")

In [ ]:
# what's the route of the line 1 (Gare de l'ouest-stockel)
df_routes[(df_routes['route_short_name']=='1')]



In [ ]:
df_stop_times

In [ ]:
# all the trips made by the line 1 over gare-de=l'ouest and stockel and viceversa (with direction 0, 1)
all_trips_metro = df_trips[(df_trips['route_id']==1)]
all_trips_metro # number of rows indicates how many trips are made in this line. 


In [370]:
all_trips_metro

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
40585,1,238683002,113603522238683002,STOCKEL,0,8968241,001m0080
40586,1,238683002,113603523238683002,STOCKEL,0,8968263,001m0080
40587,1,238683002,113603524238683002,STOCKEL,0,8968233,001m0043
40588,1,238683002,113603525238683002,STOCKEL,0,8968239,001m0080
40615,1,238683002,113603552238683002,STOCKEL,0,8968262,001m0080
...,...,...,...,...,...,...,...
47808,1,238691605,113616342238691605,GARE DE L'OUEST,1,8968497,001m0082
47809,1,238691605,113616343238691605,STOCKEL,0,8968497,001m0080
47810,1,238691605,113616344238691605,GARE DE L'OUEST,1,8968497,001m0082
47811,1,238691605,113616345238691605,STOCKEL,0,8968497,001m0080


In [461]:
def extract_trip_2(all_trips_metro, df_stop_times):
        trip_list = []
        for index, row in all_trips_metro.iterrows():
            df_trip = df_stop_times[(df_stop_times['trip_id']==row['trip_id'])]
            print(df_trip)
            start_trip = min(df_trip['arrival_time'])
            end_trip = max(df_trip['arrival_time'])
            trip_name = str(df_trip['trip_id'].unique())
            headsign = df_trip['stop_id'].iloc[-1] # headsign last value of dataframe

            #print([start_trip, end_trip])
            trip_list.append([trip_name, headsign,start_trip, end_trip])
            
        return pd.DataFrame(trip_list, columns = ['trip_name','headsign', 'start_trip', 'end_trip'])
                  

In [462]:
out = extract_trip_2(all_trips_metro,df_stop_times)
# order the trips 
trip_sorted = out.sort_values(by='start_trip')
trip_sorted


                   trip_id arrival_time departure_time stop_id  stop_sequence  \
892995  113603522238683002     23:30:39       23:30:39    8733              1   
892996  113603522238683002     23:31:36       23:31:36    8742              2   
892997  113603522238683002     23:33:17       23:33:17    8292              3   
892998  113603522238683002     23:34:29       23:34:29    8282              4   
892999  113603522238683002     23:35:55       23:35:55    8272              5   
893000  113603522238683002     23:37:01       23:37:01    8012              6   
893001  113603522238683002     23:38:22       23:38:22    8022              7   
893002  113603522238683002     23:39:19       23:39:19    8032              8   
893003  113603522238683002     23:40:25       23:40:25    8042              9   
893004  113603522238683002     23:41:40       23:41:40    8052             10   
893005  113603522238683002     23:42:40       23:42:40    8062             11   
893006  113603522238683002  

KeyboardInterrupt: 

In [441]:
trip_sorted

,trip_name,headsign,start_trip,end_trip
1204,[113573548238298504],8122,05:01:04,05:32:46
1489,[113506397238300603],8122,05:11:03,05:18:06
1201,[113564797238298504],8122,05:12:09,05:19:12
1732,[113616189238691605],8731,05:17:51,05:42:53
119,[113603774238683002],8731,05:17:53,05:43:35
...,...,...,...,...
1583,[113506496238300603],8122,24:10:44,24:30:40
1639,[113506558238300603],8731,24:15:08,24:35:23
1148,[113503915238298504],8731,24:15:08,24:35:28
1668,[113552714238300603],8731,24:32:58,24:53:13


In [ ]:
#--------------------------------------------------------------------------------------------------------------------------------------#
# Title: Import single JSON file BY Vehicle 
# Project: STIB - Data Mining 
# Author: E.O
#--------------------------------------------------------------------------------------------------------------------------------------#

# 0. -----------------  MODULES -------------------------------------------------------------------------------------------------------#
import json
import pandas as pd
import ipdb 
import datetime

# 1. ---------------- FUNCTIONS --------------------------------------------------------------------------------------------------------#

# ipdb.set_trace()  # for trace

# Function 1: Generate  pandas df for each lineId 
def create_df_lineid(df_single_line):
    """"create pandas df for a line ID"""
    
    # Initialize list
    append_data = []

    # Convert each row in df line
    for index, row in df_single_line.iterrows():
        append_data.append(pd.DataFrame(row['vehiclePositions'],index=[0]))

    # Concatenate the new data
    df_vehicle_pos = pd.concat(append_data)
    df_line_id = pd.DataFrame(df_single_line['lineId'])
    return df_vehicle_pos.join(df_line_id)


# Function 2: Preparation data from a single json file converted in pandas df.
def prepare_data(df):  
    """"Prepare the data for computation"""
    list_df = [] # initialize df
    for i in range(round(len(df)/1000) + 10):  
          
          df_data = pd.DataFrame.from_dict(df.loc[i, "data"])
          print('*******************************************************************')
          print('PROCESSING DATA: ' + str(i) + ' of ' + str(round(len(df)/1000) + 10)) 
          print('*******************************************************************')
          
          for ind in range(len(df_data)):
             df_lines = pd.DataFrame.from_dict(df_data.loc[ind, "Responses"])
             df_lines = df_lines.join(df_data['time'])
             df_lines.dropna(subset = ["time"], inplace=True) #remove nan from ts colum
        
             for j in range(len(df_lines)):
                df_lineId = pd.DataFrame.from_dict(df_lines.loc[j, 'lines'])
                df_vehicle_line_id = create_df_lineid(df_single_line = df_lineId) # create df for each vehicle line with features 
                df_time = pd.DataFrame(df_lines['time'].unique())
                df_time.columns = ['time']
                df_time = pd.to_datetime(df_time['time'], unit='ms')
                df_final = pd.concat([df_time, df_vehicle_line_id], axis = 1)
                list_df.append(df_final.head()) # list of dataframes
                #print('-------------------------------------------------------------------')
    return pd.concat(list_df)
       
                
# 2. ---------------- READING & PROCESSING ------------------------------------------------------------------------------------------------#
        
# 2.1 Opening single JSON file 
f = open('0-data/vehiclePosition02.json')
data = json.load(f)

# 2.2 Converting dictionary to Pandas DF
df = pd.DataFrame.from_dict(data)
df.head()

# Selecting just one li

# 2.3 Processing Testset 
testset = prepare_data(df)
dt_vehicle = testset[(testset['lineId']=='1')]
dt_vehicle.head()

In [442]:
dt_vehicle

,time,directionId,distanceFromPoint,pointId,lineId,dates
0,2021-09-07 07:14:54.287,8731,0,8101,1,2021-09-07
0,2021-09-07 07:14:54.287,8731,0,8011,1,2021-09-07
0,2021-09-07 07:14:54.287,8731,0,8141,1,2021-09-07
0,2021-09-07 07:14:54.287,8162,0,8072,1,2021-09-07
0,2021-09-07 07:14:54.287,8731,1,8071,1,2021-09-07
...,...,...,...,...,...,...
0,2021-09-07 07:22:22.909,8731,0,8051,1,2021-09-07
0,2021-09-07 07:22:22.909,8731,0,8741,1,2021-09-07
0,2021-09-07 07:22:22.909,8731,0,8081,1,2021-09-07
0,2021-09-07 07:22:22.909,8162,1,8122,1,2021-09-07


In [443]:
# create a new column with checking time 
trip_sorted['new_start'] = '2021-09-07 ' +  trip_sorted['start_trip'] 
trip_sorted['new_end'] = '2021-09-07 ' + trip_sorted['end_trip']

In [450]:
# categorize the trip on its trip name(id) 
def check_which_trip(dt_vehicle, trip_sorted):
    trips_check = []
    for i in range(len(dt_vehicle)):
          time_to_check = dt_vehicle.iloc[i, 0] # print first column
          real_direction = dt_vehicle.iloc[i, 1] # where the tram is going 
          current_position = dt_vehicle.iloc[i, 3] # last stop id 
          print(time_to_check)
          print('Checking the time and which trip')
          for index, row in trip_sorted.iterrows():
                
                if (row["new_start"] <= str(time_to_check)  <= row["new_end"]): 
                    #print("yes this is the trip: " +  str(row["new_start"]) + ' and' +  str(row["new_end"]))
                    trips_check.append([time_to_check, row["new_start"], row['new_end'], row['trip_name'], row['headsign'], real_direction, current_position])
                else:
                    print("not this trip")
    return pd.DataFrame(trips_check, columns = ['time_to_check', 'start_trip', 'end_trip', 'trip_name', 'headsign', 'real_direction', 'current_position'])
                    
            

In [ ]:
trips_categories  = check_which_trip(dt_vehicle, trip_sorted)

In [413]:
trips_categories['trip_name'] = pd.Series(trips_categories['trip_name'])

In [456]:
trips_categories


,time_to_check,start_trip,end_trip,trip_name,headsign,real_direction,current_position
0,2021-09-07 07:14:54.287,2021-09-07 06:49:47,2021-09-07 07:17:27,[113604209238683002],8161,8731,8101
1,2021-09-07 07:14:54.287,2021-09-07 06:49:47,2021-09-07 07:17:27,[113610664238687005],8161,8731,8101
2,2021-09-07 07:14:54.287,2021-09-07 06:52:26,2021-09-07 07:19:34,[113603894238683002],8731,8731,8101
3,2021-09-07 07:14:54.287,2021-09-07 06:52:26,2021-09-07 07:19:34,[113610349238687005],8731,8731,8101
4,2021-09-07 07:14:54.287,2021-09-07 06:54:57,2021-09-07 07:22:37,[113610663238687005],8162,8731,8101
...,...,...,...,...,...,...,...
2860,2021-09-07 07:22:22.909,2021-09-07 07:15:59,2021-09-07 07:35:48,[113503852238298504],8731,8731,8011
2861,2021-09-07 07:22:22.909,2021-09-07 07:18:16,2021-09-07 07:45:24,[113610354238687005],8731,8731,8011
2862,2021-09-07 07:22:22.909,2021-09-07 07:18:16,2021-09-07 07:45:24,[113603899238683002],8731,8731,8011
2863,2021-09-07 07:22:22.909,2021-09-07 07:20:47,2021-09-07 07:48:27,[113603824238683002],8161,8731,8011


In [459]:
# query just the headsign and real_direction equal
# now derive the distance between current position and the real direction 

# ---- Example between 8101 and 8731 -------
# Importing the geodesic module from the library
from geopy.distance import geodesic
  
# Loading coordinates
coords_1  = (1.378814, 1.529059) #coordinates 8101 
coords_2 = (1.317057, 1.535130) #coordinates 8731
  
# Print the distance calculated in meters 
print(geodesic(coords_1, coords_2).km)


6.86211556252401


In [460]:
speed = geodesic(coords_1, coords_2).km/(5/60)
speed

82.34538675028813